#Q1

In [2]:
!pip install nashpy

In [20]:
import numpy as np
import pandas as pd
import nashpy as nash

In [19]:
from pyomo.environ import *
!apt-get install glpk-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


For manual computation of saddle points we have calculated maxmin and minmax value in the matrix, if these two values are equal then saddle point exists. That value is saddle point value and the positions it is found at is saddle point.

We have constructed a LP and solved using pulp that gives the correspondig maxmin and minmax values.

In [17]:
!pip install pulp

In [ ]:
import numpy as np
import pulp

def find_saddle_points(mat):
    s_points = []
    row_min = np.min(mat, axis=1)
    col_max = np.max(mat, axis=0)
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if mat[i, j] == row_min[i] and mat[i, j] == col_max[j]:
                s_points.append((i + 1, j + 1))
    return s_points

def solve_lp(mat, maximize=True):
    r, c = mat.shape
    prob = pulp.LpProblem("Game", pulp.LpMaximize if maximize else pulp.LpMinimize)
    strat_vars = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(r if maximize else c)]
    g_val = pulp.LpVariable("g_val")

    prob += g_val
    prob += pulp.lpSum(strat_vars) == 1

    if maximize:
        for j in range(c):
            prob += g_val <= pulp.lpSum(mat[i, j] * strat_vars[i] for i in range(r))
    else:
        for i in range(r):
            prob += g_val >= pulp.lpSum(mat[i, j] * strat_vars[j] for j in range(c))

    prob.solve(pulp.PULP_CBC_CMD())
    return [v.varValue for v in strat_vars], g_val.varValue

game_mat = np.array([
    [1, 3, 8, 3],
    [7, 1, 2, -2],
    [4, 3, 12, 3]
])

print("Matrix:")
print(game_mat)

# Maximin (Row Player)
r_strat, max_val = solve_lp(game_mat, maximize=True)
print("\nRow Player's Strategy:", r_strat)
print("Maximin Value:", max_val)

# Minimax (Column Player)
c_strat, min_val = solve_lp(game_mat, maximize=False)
print("\nColumn Player's Strategy:", c_strat)
print("Minimax Value:", min_val)

# Saddle Points
s_points = find_saddle_points(game_mat)
print("\nSaddle Points:", s_points if s_points else "None")

In [12]:
import numpy as np
import nashpy as nash
game_mat = np.array([
    [1, 3, 8, 3],
    [7, 1, 2, -2],
    [4, 3, 12, 3]
])
print("Matrix:")
print(game_mat)
# Create game using Nashpy
game = nash.Game(game_mat)
equilibria = list(game.support_enumeration())
print("\nNash Equilibria:")
for eq in equilibria:
    print(f"Row Strategy: {eq[0]}, Column Strategy: {eq[1]}")
print("."*100)

Matrix:
[[ 1  3  8  3]
 [ 7  1  2 -2]
 [ 4  3 12  3]]

Nash Equilibria:
Row Strategy: [0. 0. 1.], Column Strategy: [0. 1. 0. 0.]
Row Strategy: [0. 0. 1.], Column Strategy: [0. 0. 0. 1.]
....................................................................................................


/usr/local/lib/python3.11/dist-packages/nashpy/algorithms/support_enumeration.py:260: RuntimeWarning: 
An even number of (2) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


#Q2

In [13]:
import numpy as np
import pandas as pd

def fictitious_play(payoff_matrix, iterations=1000):
    num_rows, num_cols = payoff_matrix.shape
    row_counts = np.zeros(num_rows)
    col_counts = np.zeros(num_cols)
    row_action = np.random.choice(num_rows)
    col_action = np.random.choice(num_cols)
    row_counts[row_action] += 1
    col_counts[col_action] += 1
    history = []

    for step in range(1, iterations + 1):
        row_distribution = row_counts / np.sum(row_counts)
        col_distribution = col_counts / np.sum(col_counts)
        row_payoffs = payoff_matrix.dot(col_distribution)
        col_payoffs = -payoff_matrix.T.dot(row_distribution)
        row_action = np.argmax(row_payoffs)
        col_action = np.argmax(col_payoffs)
        row_counts[row_action] += 1
        col_counts[col_action] += 1

        if step <= 50 or step > iterations - 50:
            history.append((step, row_action, col_action, row_distribution.copy(), col_distribution.copy()))

    final_row_strategy = row_counts / np.sum(row_counts)
    final_col_strategy = col_counts / np.sum(col_counts)
    game_value = final_row_strategy.dot(payoff_matrix).dot(final_col_strategy)

    return history, final_row_strategy, final_col_strategy, game_value

# Example matrix
payoff_matrix = np.array([
    [1, 3, 8, 3],
    [7, 1, 2, -2],
    [4, 3, 12, 3]
])

# Run simulation
results, final_row_strategy, final_col_strategy, game_value = fictitious_play(payoff_matrix, iterations=1000)

df = pd.DataFrame(
    results,
    columns=["Step", "Row Best Response", "Col Best Response", "Row Strategy", "Col Strategy"]
)
df["Row Strategy"] = df["Row Strategy"].apply(lambda arr: np.array2string(np.round(arr, 3), separator=','))
df["Col Strategy"] = df["Col Strategy"].apply(lambda arr: np.array2string(np.round(arr, 3), separator=','))

# Display results
print("\nFinal Results:")
print(f"Final Row Strategy: {np.round(final_row_strategy, 3)}")
print(f"Final Column Strategy: {np.round(final_col_strategy, 3)}")
print(f"Game Value: {round(game_value, 3)}")

print("\nIteration Records:")
print(df.to_string(index=False))


Final Results:
Final Row Strategy: [0.006 0.001 0.993]
Final Column Strategy: [0.005 0.001 0.    0.994]
Game Value: 3.0

Iteration Records:
 Step  Row Best Response  Col Best Response        Row Strategy              Col Strategy
    1                  0                  3          [0.,1.,0.]             [0.,1.,0.,0.]
    2                  0                  3       [0.5,0.5,0. ]         [0. ,0.5,0. ,0.5]
    3                  0                  3 [0.667,0.333,0.   ] [0.   ,0.333,0.   ,0.667]
    4                  0                  3    [0.75,0.25,0.  ]     [0.  ,0.25,0.  ,0.75]
    5                  0                  3       [0.8,0.2,0. ]         [0. ,0.2,0. ,0.8]
    6                  0                  0 [0.833,0.167,0.   ] [0.   ,0.167,0.   ,0.833]
    7                  2                  0 [0.857,0.143,0.   ] [0.143,0.143,0.   ,0.714]
    8                  2                  0 [0.75 ,0.125,0.125] [0.25 ,0.125,0.   ,0.625]
    9                  2                  0 [0.66